# Hidden Markov Models
This practice relies on training the model using the [Universal Dependencies 2.4](https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-2988#) corpus.

In [1]:
!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2988{/ud-treebanks-v2.4.tgz,/ud-documentation-v2.4.tgz,/ud-tools-v2.4.tgz}


[1/3]: https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2988/ud-treebanks-v2.4.tgz --> ud-treebanks-v2.4.tgz
--_curl_--https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2988/ud-treebanks-v2.4.tgz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  326M  100  326M    0     0   596k      0  0:09:21  0:09:21 --:--:-- 1134k  0     0   810k      0  0:06:52  0:01:08  0:05:44  459k    0  0:07:18  0:01:16  0:06:02  344k02k      0  0:07:55  0:01:41  0:06:14  483k    0   601k      0  0:09:16  0:03:49  0:05:27  495k0     0   592k      0  0:09:24  0:05:23  0:04:01  624k   613k      0  0:09:05  0:07:11  0:01:54  579k    0   589k      0  0:09:27  0:09:07  0:00:20  210k

[2/3]: https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2988/ud-documentation-v2.4.tgz --> ud-documentation-v2.4.tgz
--_curl_--https://lindat.mff.cuni.cz/repository

In [2]:
!tar -xf ud-treebanks-v2.4.tgz

Let `count`, `countB`, and `p`, respectively functions to count how many times a tag is in the corpus, how many times a tag is in the corpus given another previous one, and finally, the probability of a tag given another previous one.

In [3]:
def count(tag, corpus):
  count = 0
  for sentence in corpus:
    for token in sentence:
      if token.upos == tag:
        count+=1
  return count

def countB(tagi, tagi_1, corpus):
  count = 0
  for sentence in corpus:
    idx = 1
    while idx <= len(sentence):
      if sentence[idx-1].upos == tagi and sentence[idx-2].upos == tagi_1:
        count+=1    
      idx+=1
  return count

def p(tagi, tagi_1, corpus):
  num = countB(tagi_1, tagi, corpus)
  den = count(tagi, corpus)
  return num/den

Let's also define the training corpus and discover the tags inside it. 

Let's also build the transition matrix `tm` that gets the probability of change between tags.

In [4]:
import pyconll
import pyconll.util
import pandas

UD_ENGLISH_TRAIN = './ud-treebanks-v2.4/UD_English-LinES/en_lines-ud-train.conllu'

train = pyconll.load_from_file(UD_ENGLISH_TRAIN)

tags = set()

for sentence in train:
  for token in sentence:
    tags.add(token.upos)

tags = {tag: 0 for tag in tags}
tm = {tag: tags for tag in tags}

for rowtag in tm:
  for coltag in tm[rowtag]:
    tm[rowtag][coltag] = p(rowtag, coltag, train)

print('Transition Matrix')
print('-----------------')
print(pandas.DataFrame(tm))

Transition Matrix
-----------------
           PRON      INTJ       AUX       ADV         X       ADP     PUNCT  \
ADJ    0.008415  0.008415  0.008415  0.008415  0.008415  0.008415  0.008415   
ADP    0.235496  0.235496  0.235496  0.235496  0.235496  0.235496  0.235496   
ADV    0.026572  0.026572  0.026572  0.026572  0.026572  0.026572  0.026572   
AUX    0.064548  0.064548  0.064548  0.064548  0.064548  0.064548  0.064548   
CCONJ  0.054584  0.054584  0.054584  0.054584  0.054584  0.054584  0.054584   
DET    0.007418  0.007418  0.007418  0.007418  0.007418  0.007418  0.007418   
INTJ   0.000111  0.000111  0.000111  0.000111  0.000111  0.000111  0.000111   
NOUN   0.103521  0.103521  0.103521  0.103521  0.103521  0.103521  0.103521   
NUM    0.004539  0.004539  0.004539  0.004539  0.004539  0.004539  0.004539   
PART   0.022697  0.022697  0.022697  0.022697  0.022697  0.022697  0.022697   
PRON   0.037201  0.037201  0.037201  0.037201  0.037201  0.037201  0.037201   
PROPN  0.016275 

Let's also define two functions `tag_word_count` to tell how many times a word with a specific tag appears in the corpus, and `p_B` to define the probability of having such word with such tag in the corpus.

In [5]:
def tag_word_count(tag, word, corpus):
  count = 0
  for sentence in corpus:
    for token in sentence:
      if word == token.lemma and token.upos == tag:
        count += 1
  return count

def p_B(tag, word, corpus):
  num = tag_word_count(tag, word, corpus)
  den = count(tag, corpus)
  return num/den

Having the training functions, let's define `ctrl_phrase` as a testing phrase to check its B matrix.

In [6]:
ctrl_phrase = 'you will back the bill'.split()

mb = {word: {tag: 0 for tag in tags} for word in ctrl_phrase}

for word in ctrl_phrase:
  for sentence in train:
    for token in sentence:
      if token.lemma == word:
        mb[word][token.upos] = p_B(token.upos, word, train)

print(pandas.DataFrame(mb))

            you      will      back       the      bill
ADJ    0.000000  0.000000  0.000000  0.000000  0.000000
ADP    0.000000  0.000000  0.000000  0.000000  0.000000
ADV    0.000000  0.000000  0.024531  0.000722  0.000000
AUX    0.000000  0.052336  0.000000  0.000000  0.000000
CCONJ  0.000000  0.000000  0.000000  0.000000  0.000000
DET    0.000000  0.000000  0.000000  0.591573  0.000000
INTJ   0.000000  0.000000  0.000000  0.000000  0.000000
NOUN   0.000000  0.000443  0.001993  0.000000  0.000111
NUM    0.000000  0.000000  0.000000  0.000000  0.000000
PART   0.000000  0.000000  0.000000  0.000000  0.000000
PRON   0.097779  0.000000  0.000000  0.000000  0.000000
PROPN  0.000000  0.000000  0.000000  0.000000  0.000599
PUNCT  0.000000  0.000000  0.000000  0.000000  0.000000
SCONJ  0.000000  0.000000  0.000000  0.000000  0.000000
SYM    0.000000  0.000000  0.000000  0.000000  0.000000
VERB   0.000000  0.000169  0.000507  0.000000  0.000000
X      0.000000  0.000000  0.000000  0.000000  0

Finally, let's define a list of dictionaries `hmm` to save the Hidden Markov Model for this case.

In [7]:
hmm = []
for word in mb:
  argmax = {'p': 0, 'upos': '', 'word': word}
  for tag in mb[word]:
    if mb[word][tag] > argmax['p']:
      argmax = {'p': mb[word][tag], 'upos': tag, 'word': word}
  hmm.append(argmax)

for e in hmm:
  print(e)

{'p': 0.09777870043595599, 'upos': 'PRON', 'word': 'you'}
{'p': 0.052336448598130844, 'upos': 'AUX', 'word': 'will'}
{'p': 0.024531024531024532, 'upos': 'ADV', 'word': 'back'}
{'p': 0.591572799332499, 'upos': 'DET', 'word': 'the'}
{'p': 0.0005991611743559018, 'upos': 'PROPN', 'word': 'bill'}
